In [2]:
import pandas as pd
df = pd.read_hdf('outputs/budzety_clean_topics_bert.h5', key='df')

# Pomysły co robić dalej, problemy, wyzwania, itp...

***

## Wybór odpowiedniej metody reprezentacji tematów

1. BERTopic  
2. LDA  
3. Top2Vec  
4. inna...

***

## Różna ilość tematów

Modyfikujemy zmienną `N_TOPICS` i patrzymy jakie tematy otrzymujemy.  
Proponuję wybierać większą liczbę tematów niż proponowane 20.

***

## Grupowanie tematów

Utworzyć cechę `topics_groups` i poprzypisywać do niej poszczególne tematy. Tzn. grupa zielona to tematy 12, 5, 4 i 6, grupa czerwona to tematy 17, 8, 13, 1, 14 itp.  
Grupy mogą mieć jakieś nazwy (podejście subiektywne) np. zieleń, transport, sport...
Grupowanie tematów nie musi być robione w odniesieniu do kolorów. Można wejść na wyższy poziom klastrowania hierachicznego i np. podzielić je względem konkrentego poziomu  
Np. pierwszy podział klastrowania to podział na grupy, do którtych należą tematy 15 12 5 4 6 17 8 13 1 14 (pierwsza grupa) i pozostałe tematy (jako druga grupa). Więc tworzymy tylko 2 grupy.

In [7]:
df[['nazwa', 'topics']]

,nazwa,topics
0,Projektistatus,-1
1,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,3
2,Cykliczne warsztaty dla seniorów z dzielnicy O...,1
3,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",8
4,Montaż słupów oświetleniowych na chodniku przy...,-1
...,...,...
23025,Zielony skwer na Zabtociu,13
23026,Znani Biatopradniczanie dla Mieszkanców,2
23027,ZRÓBMY SKWER PRZY FABRYCZNEJ! Edukacja i kultu...,11
23028,Zróbmy sobie Park,15


![title](images/topics.png)

***

## Rozbicie algorytmu Bertopic

Rozbijamy Bertopic na UMAP i HDBSCAN i próbujemy różnych parametrów tych dwóch algorytmów.

In [ ]:
# UMAP - zmniejszenie ilości wymiarów
umap_model = umap.UMAP(n_neighbors=15,
                       n_components=10,
                       min_dist=0.0,
                       metric='cosine',
                       low_memory=False)

# HDBSAN - klasteryzacja dokumentów
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10,
                                min_samples=1,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True)

In [ ]:
# Bertopic
model = BERTopic(top_n_words=20,
                       n_gram_range=(1,2), 
                       calculate_probabilities=True,
                       umap_model= umap_model,
                       hdbscan_model=hdbscan_model,
                       similarity_threshold_merging=0.5,
                       verbose=True,
                       language='polish')

topics, probs = model.fit_transform(docs)
# lub 
# topics, probs = model.fit_transform(docs, embeddings)
# jeżeli bierzemy pod uwagę embeddingi (patrz punkt poniżej)

***

## Użycie różnych embeddingów

Tokeny mogą być różnie reprezentowane (embedding)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=5)
embeddings = vectorizer.fit_transform(docs)
topics, probs = model.fit_transform(docs, embeddings)

# lub

from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distiluse-base-multilingual-cased")
embeddings = sentence_model.encode(docs, show_progress_bar=False)
topics, probs = model.fit_transform(docs, embeddings)

Istnieje całe mnóstwo reprezentacji wektorowych, każda z nich może mieć dodatkowo swoje własne parametry

***

## Badanie trendów zmian

In [11]:
def most_common_topics(city: str, year: str, top: int = 3) -> pd.Series:
  result = df[(df['miasto'] == city) & (df['rok'] == year)]['topics'].value_counts()[1:top+1]
  return result

Wybieramy miasto i patrzymy jakie były najpopularniejsze tematy na przestrzen lat.   
(Domyślnie 3 tematy z pominięciem szumu)

In [12]:
most_common_topics('GDYNIA', '2018')

0     9
14    8
8     5
Name: topics, dtype: int64

In [13]:
most_common_topics('GDYNIA', '2018/19')

Series([], Name: topics, dtype: int64)

In [14]:
most_common_topics('GDYNIA', '2019')

0    44
3    17
1    11
Name: topics, dtype: int64

In [15]:
most_common_topics('GDYNIA', '2020')

0    39
1    24
3    18
Name: topics, dtype: int64

Widzimy, że w Gdyni najpopularniejsze były tematy `0 8 14`, potem `0 1 3` i rok później również `0 1 3`. To już jest podstawą do utworzenia pewnej historii.

Możemy nie patrzeć na zmiany tematów na przestrzeni lat, ale na zmiany w grupach tematów na przestrzeni lat.

Jeżeli mamy już informację o zmianie tematów na przestrzeni lat dla danego miasta to możemy sprawdzić z czego ona wynika.
Czy na tą zmianę miało wpływ to, że:
* miasta o podobnej liczbie ludności do wybranego wykazywały podobne trendy zmian na przestrzeni lat 
* miasta o podobnej powierzchni do wybranego wyjazywały podobne trendy zmian na przestrzeni lat
* miasta sąsiadujące z wybranym miastem i ich najpopularniejsze tematy wpłynęły na zmianę w tematach wybranego miasta
na przestrzeni lat
* czy miasta o podobnym budżecie przeliczonym na 100 tys. mieszkańców jak nasze wybrane miasto "interesowały" się podobnym tematami

***

## Problemy do rozpracowania ::: Sprawdzenie poprawności klasyfikacji tematów

Nie wszystkie tematy zostały dobrze sklasyfikowane

In [23]:
pd.set_option('max_colwidth', 400)
pd.DataFrame(df[['rok', 'miasto', 'topics', 'topics_desc', 'nazwa', 'names']].iloc[18589]).T

,rok,miasto,topics,topics_desc,nazwa,names
18589,2018,WARSZAWA,12,"[pies, psi, ursynów, pszczoła, komar, ursynowie, owad, nietoperz, zwierzę, wiewiórka]",Scrabble na Ursynowie,scrabble ursynowie


Np. zauważyłem, że dla Ursynowa bardzo dużo wpada do kategorii `zwierzęcej` (temat nr 12)

In [50]:
df[df['nazwa'].str.contains('Ursyn')][['topics', 'topics_desc', 'nazwa']]

,topics,topics_desc,nazwa
17953,12,"[pies, psi, ursynów, pszczoła, komar, ursynowie, owad, nietoperz, zwierzę, wiewiórka]",Doświetlenie przejść dla pieszych na Ursynowie Południowym
17967,7,"[chodnik, warszawa, warszawski, warszawo, park, polski, budynek, miejsce, sąsiedzki, poznański]","Pij, Warszawo (Ursynów Południowy)"
17977,-1,"[chodnik, park, ulica, budowa, warsztat, koncert, miejsce, dzielnica, modernizacja, muzyczny]",Ursynowskie Murale
18022,12,"[pies, psi, ursynów, pszczoła, komar, ursynowie, owad, nietoperz, zwierzę, wiewiórka]",Doświetlenie przejść dla pieszych na Ursynowie Północnym
18069,14,"[cmentarz, kościół, pomnik, rewitalizacja, rzeźba, wieża, chodnik, ulica, obserwatorium, widokowy]",Ursynowskie rzeźby po 40-tu latach do renowacji
...,...,...,...
22108,12,"[pies, psi, ursynów, pszczoła, komar, ursynowie, owad, nietoperz, zwierzę, wiewiórka]",Neon na budynku Dzielnicowego Ośrodka Kultury Ursynów
22112,12,"[pies, psi, ursynów, pszczoła, komar, ursynowie, owad, nietoperz, zwierzę, wiewiórka]",Doświetlenie przejść dla pieszych na Ursynowie
22114,12,"[pies, psi, ursynów, pszczoła, komar, ursynowie, owad, nietoperz, zwierzę, wiewiórka]",Bezpłatne zajęcia dla dzieci i młodzieży w Dzielnicowym Ośrodku Kultury Ursynów
22115,-1,"[chodnik, park, ulica, budowa, warsztat, koncert, miejsce, dzielnica, modernizacja, muzyczny]",Letnie koncerty na Ursynowie


Chyba przyporządkowuje `Ursynów` jako `urson`

![title](images/urson.png)  
źródło: https://pl.wikipedia.org/wiki/Urson_ameryka%C5%84ski

Podobnie wiele tematów z miasta Łódź jest błędnie przypisywanych do kategorii `morsko-rzecznej`

In [54]:
df[df['nazwa'].str.contains('Łodzi')][['topics', 'topics_desc', 'nazwa']]

,topics,topics_desc,nazwa
11504,15,"[łódź, plaża, plażowy, łódzki, morski, kajakowy, łódka, kajak, łodzianin, pomorski]",Mistrzostwa Miasta Gorzowa w Smoczych Łodziach
11961,-1,"[chodnik, park, ulica, budowa, warsztat, koncert, miejsce, dzielnica, modernizacja, muzyczny]",PLAŻA W ARTURÓWKU- miejscem wypoczynku i rekreacji dla Łodzian.
12011,15,"[łódź, plaża, plażowy, łódzki, morski, kajakowy, łódka, kajak, łodzianin, pomorski]",Kino plenerowe - letnie w Arturówku - miejscem wypoczynku i rekreacji dla Łodzian. Bliżej kultury na łonie natury.
12057,15,"[łódź, plaża, plażowy, łódzki, morski, kajakowy, łódka, kajak, łodzianin, pomorski]",Drzwi do przyszłości - kolorowe i bezpieczne wejście do sal zabaw i nauki dla najmłodszych Łodzian.
12167,-1,"[chodnik, park, ulica, budowa, warsztat, koncert, miejsce, dzielnica, modernizacja, muzyczny]","Budowa boiska wielofunkcyjnego sportowego o sztucznej nawierzchni przy SP 113 w Łodzi, ul. Unicka 6."
...,...,...,...
14974,15,"[łódź, plaża, plażowy, łódzki, morski, kajakowy, łódka, kajak, łodzianin, pomorski]","Bezpieczna 4-stka monitoring wizyjny wokół szkoły podstawowej nr 4 w Łodzi, ul. Milionowa"
14981,-1,"[chodnik, park, ulica, budowa, warsztat, koncert, miejsce, dzielnica, modernizacja, muzyczny]",Przywróćmy blask infrastrukturze sportowej w XXIX LO w Łodzi.
14984,15,"[łódź, plaża, plażowy, łódzki, morski, kajakowy, łódka, kajak, łodzianin, pomorski]","Remont szatni w budynku Szkoły Podstawowej nr 64 w Łodzi, ul. Wróblewskiego 6"
14985,15,"[łódź, plaża, plażowy, łódzki, morski, kajakowy, łódka, kajak, łodzianin, pomorski]",Sala językowa dla uczniów Szkoły Podstawowej nr 64 w Łodzi.
